In [ ]:
!pip install pyodbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 8.2 MB/s eta 0:00:00


In [ ]:
import pyodbc
import requests  # Import requests library for making HTTP requests

# Database connection function

In [ ]:
def connect_db():
    server = 'your_server_name'
    database = 'your_database_name'
    username = 'your_username'
    password = 'your_password'
    driver= '{ODBC Driver 18 for SQL Server}'  # Update driver if necessary
    connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    connection = pyodbc.connect(connection_string)
    return connection

# Ensure the Books table exists

In [ ]:
def setup_table(connection):
    cursor = connection.cursor()
    create_table_sql = """
    IF OBJECT_ID('dbo.Books', 'U') IS NULL
    BEGIN
        CREATE TABLE dbo.Books (
            BookID INT IDENTITY(1,1) PRIMARY KEY,
            Title NVARCHAR(255) NOT NULL,
            Author NVARCHAR(255) NOT NULL,
            Genre NVARCHAR(100),
            PublicationYear INT
        );
    END;
    """
    cursor.execute(create_table_sql)
    connection.commit()
    print("Books table created (or already exists).")
    cursor.close()

# Close the database connection

In [ ]:
def disconnect_db(connection):
    connection.close()
    print("Database connection closed.")

# Fetch books from Google Books API

In [ ]:
def fetch_books_from_google_api(query):
    api_url = f"https://www.googleapis.com/books/v1/volumes?q={query}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        books = []
        for item in data['items'][:5]:  # Limit to 5 results for demo purposes
            title = item['volumeInfo'].get('title', 'N/A')
            authors = ', '.join(item['volumeInfo'].get('authors', ['Unknown Author']))
            genre = item['volumeInfo'].get('categories', ['N/A'])[0]
            publication_year = item['volumeInfo'].get('publishedDate', 'N/A')[:4]
            books.append((title, authors, genre, int(publication_year) if publication_year.isdigit() else None))
        return books
    else:
        print("Error fetching data from Google Books API.")
        return []

# Insert fetched books into the database

In [ ]:
def insert_fetched_books(connection, books):
    cursor = connection.cursor()
    insert_sql = "INSERT INTO dbo.Books (Title, Author, Genre, PublicationYear) VALUES (?, ?, ?, ?);"
    for book in books:
        cursor.execute(insert_sql, book)
    connection.commit()
    print("Fetched books from Google API inserted into Books table.")
    cursor.close()

# Retrieve data from the database

In [ ]:
def retrieve_data(connection):
    cursor = connection.cursor()
    select_sql = "SELECT * FROM dbo.Books"
    cursor.execute(select_sql)
    rows = cursor.fetchall()
    print("Retrieved data from the Books table:")
    for row in rows:
        print(f"ID: {row.BookID}, Title: {row.Title}, Author: {row.Author}, Genre: {row.Genre}, Year: {row.PublicationYear}")
    cursor.close()

# Update data from the database

In [ ]:
def update_book(connection, book_id, new_title=None, new_author=None, new_genre=None, new_year=None):
    cursor = connection.cursor()
    update_sql = "UPDATE dbo.Books SET "
    update_fields = []
    values = []

    if new_title:
        update_fields.append("Title = ?")
        values.append(new_title)
    if new_author:
        update_fields.append("Author = ?")
        values.append(new_author)
    if new_genre:
        update_fields.append("Genre = ?")
        values.append(new_genre)
    if new_year:
        update_fields.append("PublicationYear = ?")
        values.append(new_year)

    if update_fields:
        update_sql += ", ".join(update_fields)
        update_sql += " WHERE BookID = ?"
        values.append(book_id)

        cursor.execute(update_sql, values)
        connection.commit()
        print(f"Book with ID {book_id} updated successfully.")
    else:
        print("No fields provided for update.")
    cursor.close()

# Delete sample data from the database

In [ ]:
def delete_sample_data(connection):
    cursor = connection.cursor()
    delete_sql = "DELETE FROM Books WHERE Author = 'Sample Author';"
    cursor.execute(delete_sql)
    connection.commit()
    print("Sample data deleted.")
    cursor.close()

# Main test script

In [ ]:
if __name__ == "__main__":
    conn = connect_db()
    setup_table(conn)
    books = fetch_books_from_google_api("Python programming")
    insert_fetched_books(conn, books)
    update_book(conn, 1, new_title="Updated Python Book")
    retrieve_data(conn)
    delete_sample_data(conn)
    disconnect_db(conn)

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 18 for SQL Server' : file not found (0) (SQLDriverConnect)")